# Search for Trump and Biden political ads

1. we will use API of [FCC's Public Inspection File](https://publicfiles.fcc.gov/developer) to:
* collect dataset of tv and radio facilities
* search for Trump and Biden related files in such facilities  

2. We will upload pdf files to AWS s3 bucket
3. We will retrieve page images from pdf files and upload them to s3 bucket
4. We will retrieve text where possible and upload text files to s3 bucket
5. We will OCR those pdf files from which we failed to retrieve text and will put missing text files to s3 bucket

# Initializer

In [1]:
!pip install pdfplumber
!pip install boto3
from IPython.display import clear_output
clear_output()


In [2]:
import requests
import pandas as pd
import pyarrow
from io import BytesIO
import pyarrow.parquet as pq
import boto3
import json

## AWS access keys
you will need your AWS account access keys to upload files to s3 bucket

In [3]:
from google.colab import userdata

AWS_BRG_ACCESS_KEY = userdata.get('AWS_BRG_ACCESS_KEY')
AWS_BRG_SECRET_ACCESS_KEY = userdata.get('AWS_BRG_SECRET_ACCESS_KEY')


s3_client = boto3.client('s3',
            aws_access_key_id = AWS_BRG_ACCESS_KEY,
            aws_secret_access_key = AWS_BRG_SECRET_ACCESS_KEY)

# create tv_facilities df

We are going to create a Pandas dataframe with id and other data for the following 4 types of facilities:
* tv
* fm
* am
* cable

We are primarily interested in the field 'id', since we will be using it to search for pdf files

[link to API docs](https://publicfiles.fcc.gov/!/relationship/!/search/!/facility/!/service/get_serviceType_facility_getall#!/service/get_serviceType_facility_getall)

In [4]:
facilities_df = pd.DataFrame()

for facility_type in ['tv','fm','am','cable']:
  url=f'https://publicfiles.fcc.gov/api/service/{facility_type}/facility/getall'
  parameters = {'serviceType':facility_type,
                'format':'json'}
  r = requests.get(url, params=parameters)

  if r.status_code==200:
    results = r.json()['results']
    for key, v in results.items():

      if v:

        df = pd.DataFrame(v)
        df['facility_type'] = facility_type

        # else:
        print(f"{facility_type}: {len(df)}")
        facilities_df  = pd.concat([facilities_df, df])

facilities_df.dropna(subset=['id','facility_type','callSign'],
                     inplace=True,
                     ignore_index=True)

facilities_df.shape

tv: 2228
fm: 11478
am: 4693
cable: 3936


(18399, 57)

In [4]:
facilities_df

,id,callSign,frequency,activeInd,facility_type,cableSystemId,headEndId,operatorId,legalName,applicationId,...,localFileContactFax,localFileContactPhone,localFileContactEmail,accessToken,principalAddressInLocalFiles,cableServiceZipCodes,cableServiceEmpUnits,cableCommunities,cenemail,cenphone
0,127882,KBKF-LD,,Y,tv,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,776239,KKAD,,Y,tv,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22590,WTVC,186.0,Y,tv,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,776230,KKAC,,Y,tv,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,776228,KKEL,,Y,tv,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18394,61643,WSNR,620.0,Y,am,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18395,14763,KCLI,1320.0,Y,am,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18396,35857,KVLI,1140.0,Y,am,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18397,87177,KFSG,1690.0,Y,am,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# trump_df, biden_df: indices of pdf files

## read existing files

In [5]:
url = 'https://github.com/aguille-vert/trump-biden-ads/raw/main/data/biden_df.parquet'

# Download the file content
response = requests.get(url)
file_content = BytesIO(response.content)

# Read the Parquet file into a pandas DataFrame
table = pq.read_table(file_content)
biden_df = table.to_pandas()

biden_df.shape

(29613, 10)

In [20]:
biden_df

,file_id,file_name,file_extension,file_size,file_status,file_folder_path,folder_id,file_manager_id,create_ts,last_update_ts
0,77bfaf7e-21b0-61bf-386f-fba767880092,KVCW BIDEN 6090469 INV,pdf,292386,com_prc,Political Files/2024/Federal/President/Biden f...,75920302-e571-14a8-d438-30dec2649f78,dbdd84c1-323a-4c80-8d7d-bc006b973261,2024-05-24 14:21:53-04:00,2024-05-24 14:21:56-04:00
1,a59b2440-f8a8-02e0-d0cb-3e1ceda984a1,KVCW BIDEN 6089194 INV,pdf,289714,com_prc,Political Files/2024/Federal/President/Biden f...,75920302-e571-14a8-d438-30dec2649f78,0edea95d-21a5-447d-ba99-14c648559b34,2024-05-24 14:21:53-04:00,2024-05-24 14:21:55-04:00
2,fce785a4-3faa-7b5d-9c25-eb3b4fe72c39,NVCW BIDEN 6090480 INV,pdf,292021,com_prc,Political Files/2024/Federal/President/1NVCW/B...,4b4be962-7dcb-4e1f-c0c6-312eefb9a4c7,b6efab65-ea8a-4cef-a139-6d40957bdff1,2024-05-24 14:18:06-04:00,2024-05-24 14:18:08-04:00
3,62a8840d-f10e-6d1e-03dc-bc102681b134,NVCW BIDEN 6089186 INV,pdf,282914,com_prc,Political Files/2024/Federal/President/1NVCW/B...,4b4be962-7dcb-4e1f-c0c6-312eefb9a4c7,4ff8d0ad-4cd6-4b6b-907a-69cb6843172c,2024-05-24 14:18:06-04:00,2024-05-24 14:18:08-04:00
4,8332f3bf-405d-4b92-b670-fa300053c7f2,NVCW BIDEN 6089185 INV,pdf,284497,com_prc,Political Files/2024/Federal/President/1NVCW/B...,4b4be962-7dcb-4e1f-c0c6-312eefb9a4c7,f14e86a8-e2d2-454d-b514-f903c92fbbd1,2024-05-24 14:18:06-04:00,2024-05-24 14:18:08-04:00
...,...,...,...,...,...,...,...,...,...,...
29608,c279239d-737b-4660-0e25-b0bf58a5dece,Joe Biden for President 8.20.19-8.26.19 Contract,pdf,50961,com_cpy,Political Files/2019/Federal/President/Joe Bid...,a564dbc1-7f75-b5b3-08cb-4a612ddbcf56,7472cddc-b32e-4da3-ac18-1392d83a3c8c,2019-08-19 15:08:46-04:00,2019-08-19 15:09:01-04:00
29609,7e1eeef4-f23f-b723-44d1-05c75a0ebf3b,Joe Biden for President-KCWI NAB #2116749 0820...,pdf,98744,com_cpy,Political Files/2019/Federal/President/Joe Bid...,e66dcbef-b04b-6e46-94cd-04798673cbc4,56c1e2d6-7295-4c39-a2d0-67f9da3d8f4f,2019-08-19 14:03:46-04:00,2019-08-19 14:04:00-04:00
29610,374def10-7e02-eedd-6479-a8a86790efb9,Joe Biden for President-KCWI #2116749 082019-0...,pdf,218203,com_cpy,Political Files/2019/Federal/President/Joe Bid...,e66dcbef-b04b-6e46-94cd-04798673cbc4,21c1a535-d163-493c-9cf4-2eeac8380104,2019-08-19 14:03:46-04:00,2019-08-19 14:03:53-04:00
29611,4990e9e6-7175-117e-d1e2-731df6b75f42,Joe Biden for President-WOI NAB #2116768 08201...,pdf,97334,com_cpy,Political Files/2019/Federal/President/Joe Bid...,488bd9ec-ce28-53b5-03ba-4f20336de02f,00e709d5-c39c-4dd8-8cd4-d120f5bab721,2019-08-19 13:59:15-04:00,2019-08-19 13:59:30-04:00


In [6]:
url='https://github.com/aguille-vert/trump-biden-ads/raw/main/data/trump_df.parquet'

# Download the file content
response = requests.get(url)
file_content = BytesIO(response.content)

# Read the Parquet file into a pandas DataFrame
table = pq.read_table(file_content)
trump_df = table.to_pandas()

trump_df.shape

(11458, 12)

In [ ]:
trump_df

,file_id,file_name,file_extension,file_size,file_status,file_folder_path,folder_id,file_manager_id,create_ts,last_update_ts,tv_id,tv_callSign
0,27b8fab9-298c-75a5-eca2-8965a97f9879,Donald Trump for Pres Est 11107 March Invoice ...,pdf,217575,com_prc,Political Files/2024/Federal/President/Donald ...,cf34b0ed-344b-fdf8-d16d-c540ec2d3f17,86546c9f-5aa9-4732-bf3d-b66c1a13d767,2024-05-13 12:16:05-04:00,2024-05-13 12:16:08-04:00,72300,WHNS
1,8e5549db-b75f-37a0-09d8-9bd5110c31e7,MAGA Inc. NAB PB-19 3.5.24,pdf,1061164,com_prc,Political Files/2024/Federal/President/Donald ...,1278eafd-1796-892e-6e11-6a8bf01d39ce,6741d2b1-91d3-43ea-bd3e-d529dd4eca26,2024-04-22 17:40:25-04:00,2024-04-22 17:40:27-04:00,66469,KFSM-TV
2,2da2f688-44de-a335-a1d8-26666d430f05,Note2 RE PA Primary Spot,pdf,31529,com_prc,Political Files/2024/Federal/President/Donald ...,1278eafd-1796-892e-6e11-6a8bf01d39ce,74f49029-21c1-4d67-892d-94a1136b1b1e,2024-04-22 17:40:24-04:00,2024-04-22 17:40:25-04:00,66469,KFSM-TV
3,f2d03ba5-5416-8659-5f7e-7368fd99e98d,Strategic Media Serv423827-1,pdf,224890,com_prc,Political Files/2024/Federal/President/Donald ...,a4d2c087-5fc6-a9b5-24c6-e5d23189d69c,26110d19-7cbe-4852-9df3-737029ec213c,2024-04-15 10:41:57-04:00,2024-04-15 10:41:59-04:00,60963,WOLO-TV
4,090d96df-45c0-d65b-f32b-7414d0ed2db4,Donald Trump WCSH2843077--1 (1.16-1.23),pdf,268068,com_prc,Political Files/2024/Federal/President/Donald ...,173ce546-0cc7-82c4-e39a-8426f69c0324,2c79184e-4324-43fe-a5a4-03904184b464,2024-04-11 14:45:58-04:00,2024-04-11 15:00:00-04:00,39664,WCSH
...,...,...,...,...,...,...,...,...,...,...,...,...
11453,2e96c9d7-6c2a-d30e-a961-9608455f4bd4,Donald Trump on SNL Aired on 11-7-15 Notice,docx,213749,com_etl,Political Files/2015/Federal/President/Donald ...,c3e2a4ab-b52c-44cf-3dfb-429bcb88b819,2e96c9d7-6c2a-d30e-a961-9608455f4bd4,2015-11-09 17:37:00-04:00,2015-11-09 17:37:00-04:00,74449,KSWT
11454,e76d93a3-3bf2-c9bc-eb9a-b83dfe2b78f9,Donald-Trump-SaturdayNightLive-November7th,pdf,246530,com_etl,Political Files/2015/Federal/President/Donald ...,0de15c77-a895-b36a-6af2-dde5b9ac4bc1,e76d93a3-3bf2-c9bc-eb9a-b83dfe2b78f9,2015-11-09 13:23:04-04:00,2015-11-09 13:23:04-04:00,12427,KNDU
11455,46c56fd6-e719-ab5f-9c22-03df94659f75,Donald-Trump-SaturdayNightLive-November7th,pdf,246920,com_etl,Political Files/2015/Federal/President/Donald ...,64a49bac-fef6-6b21-75f8-6094cd9c01cb,46c56fd6-e719-ab5f-9c22-03df94659f75,2015-11-09 13:22:09-04:00,2015-11-09 13:22:09-04:00,12395,KNDO
11456,f22365a4-e3c6-5e71-49c3-9ba302289a11,Donald Trump Saturday Night Live November 7 2015,pdf,144905,com_etl,Political Files/2015/Federal/President/Donald ...,1cba2d4e-c478-efb8-bbf3-e02f1d9f636d,f22365a4-e3c6-5e71-49c3-9ba302289a11,2015-11-09 11:45:45-04:00,2015-11-09 11:45:45-04:00,65583,KOMU-TV


# search for pdf files

running cells in this section takes multiple hours. They should be run to update the files biden_df and trump_df above

We will be searching FCC database for the following keywords:
* trump
* biden

[link to API docs](https://publicfiles.fcc.gov/!/relationship/!/search/!/facility/!/service/!/search/!/search/get_search_key_searchKey_format)

## search for 'trump'

In [ ]:
q='trump'
trump_df = pd.DataFrame()
for row  in facilities_df.itertuples():
  try:
    entid = row.id
    print(row[0], entid)
    url = f"https://publicfiles.fcc.gov/api/manager/search/key/{q}.json?entityId={entid}"
    r = requests.get(url)
    df = pd.DataFrame(r.json()['searchResult']['files'])
    tv_id = row.id
    tv_name = row.callSign
    df['tv_id'] = tv_id
    df['tv_callSign'] = tv_name
    if df.shape[0] > 0:
      trump_df = pd.concat([trump_df,
                            df])
      print(row[0],entid, df.shape, trump_df.shape)
  except:
    pass

## search for 'biden

In [ ]:
q='biden'
biden_df = pd.DataFrame()
for row  in facilities_df.itertuples():
  try:
    entid = row.id
    print(row[0], entid)
    url = f"https://publicfiles.fcc.gov/api/manager/search/key/{q}.json?entityId={entid}"
    r = requests.get(url)
    df = pd.DataFrame(r.json()['searchResult']['files'])
    tv_id = row.id
    tv_name = row.callSign
    df['tv_id'] = tv_id
    df['tv_callSign'] = tv_name
    if df.shape[0] > 0:
      biden_df = pd.concat([biden_df,
                            df],ignore_index=True)
      print(row[0],entid, df.shape, biden_df.shape)
  except:
    pass

# pre-process pdf files for storage

## functions

In [9]:
import requests
from io import BytesIO
import pdfplumber

def download_pdf_file_to_buffer(
                                 folder_id,
                                 file_manager_id
                                ):
  url = f"https://publicfiles.fcc.gov/api/manager/download/{folder_id}/{file_manager_id}.pdf"
  r = requests.get(url)
  return BytesIO(r.content)

def extract_words_images_from_pdf_s3(buffer,
                                      IMAGE_RESOLUTION=150):

  """
  Extracts words and images from pdf file; resizes images to IMAGE_WIDTH
  """
  buffer.seek(0)
  extracted_words = {}
  extracted_images = {}
  with pdfplumber.open(buffer) as pdf:

    for page_num, pdf_page in enumerate(pdf.pages):
      try:
          page_extracted_words = pdf_page.extract_words()
          extracted_words[page_num] = [
              {
                  'text': i['text'],
                  'top': i['top'],
                  'bottom': i['bottom'],
                  'x0': i['x0'],
                  'x1': i['x1']
              }
              for i in page_extracted_words
          ]
      except:
        pass

      try:
          page_image = pdf_page.to_image(resolution=IMAGE_RESOLUTION)
          pil_image = page_image.original
          if pil_image:
            extracted_images[page_num] = pil_image
      except:
        pass

  return extracted_words, extracted_images

loop over biden_df or trump_df file and put the following types of files into s3 bucket:
* original pdf file
* file with page-by-page text extracted from pdf
* image files, with page-by-page images extracted from pdf

In [10]:

pdf_files = []
extracted_images = []
extracted_words = []

bucket = <INSERT YOUR BUCKET NAME HERE>
for row in biden_df.itertuples():
  try:
    url = f"https://publicfiles.fcc.gov/api/manager/download/{row.folder_id}/{row.file_manager_id}.pdf"
    r = requests.get(url, timeout = 30)
    buffer = BytesIO(r.content)
    words, images = extract_words_images_from_pdf_s3(buffer,
                                      IMAGE_RESOLUTION=150)
    extracted_words.append((row.file_name, words))
    extracted_images.append((row.file_name, images))

    print("uploaded ", row.file_name, row[0])
    key = f"FCC/pdfs/{row.file_name}/doc.pdf"
    invoice_type = 'text' if len(words)==0 else 'image'
    pdf_files.append((row.file_name, invoice_type))




  except:
    print("exception:", row[0], row.file_name)

  if row[0]%10==0 and row[0]!=0:
    print(row)
    print(f"started uploading {len(extracted_words)} text files")
    extracted_words_df = pd.DataFrame(extracted_words,
                                      columns=['file_name',
                                      'extracted_words'])
    extracted_words_df['invoice_type'] = extracted_words_df['extracted_words'].\
                              apply(lambda x: 'image' if len(x)==0 else 'text')

    for row in extracted_words_df.itertuples():
      if row.invoice_type=='text':
        key = f'FCC/extracted_texts/{row.file_name}/extracted_text.json'
        s3_client.put_object(Bucket=bucket,
                            Key=key,
                            Body = json.dumps(row.extracted_words))

    print(f"started uploading {len(extracted_images)} image files")
    for file_name, images in extracted_images:
      for page_num, image in images.items():
        key = f"FCC/images/{file_name}/page_{page_num}.jpg"
        print(key)
        buffer=BytesIO()
        image.save(buffer,format='JPEG')
        s3_client.put_object(Bucket=bucket,
                            Key = key,
                            Body = buffer.getvalue())
    break
    extracted_words = []
    extracted_images = []

exception: 0 KVCW BIDEN 6090469 INV
uploaded  KVCW BIDEN 6089194 INV 1
uploaded  NVCW BIDEN 6090480 INV 2
exception: 3 NVCW BIDEN 6089186 INV
exception: 4 NVCW BIDEN 6089185 INV
uploaded  Biden Invoice 2909154-1 5
uploaded  Biden-for-President-Political-PFD-05-24-2024 6
exception: 7 KSNV BIDEN 6090455 INV
exception: 8 KSNV BIDEN 6084304 INV 2
exception: 9 KSNV BIDEN 6089179 INV
uploaded  KSNV BIDEN 6089190 INV 10
Pandas(Index=10, file_id='c82213d4-fe46-acf7-44cb-8e0054dfef8b', file_name='KSNV BIDEN 6089190 INV', file_extension='pdf', file_size='307715', file_status='com_prc', file_folder_path='Political Files/2024/Federal/President/Biden for President', folder_id='922a7ee7-6797-0db6-8a4a-e5138ce1a0e7', file_manager_id='ee7325a0-bf77-4acc-a9b0-b7e7a00853cf', create_ts=Timestamp('2024-05-24 13:16:35-0400', tz='pytz.FixedOffset(-240)'), last_update_ts=Timestamp('2024-05-24 13:16:36-0400', tz='pytz.FixedOffset(-240)'))
started uploading 5 extracted words
started uploading 5 images
FCC/imag